In [22]:
import re

T_INIT = "init"
T_KEYWORD = "keyword"
T_FUNCTION = "function"
T_CLASS = "class"
T_OP = "op"
T_INT = "int"
T_FLOAT = "float"
T_STRING = "string"
T_BOOL = "bool"
T_ID = "id"
T_IF_ELSE = "if_else"
T_WHILE = "while"
T_EOF = "eof"
T_DELIMITER = "delimiter"
T_BLOCK = "block"
T_PARA = "parenthesis"
T_RETURN = "return"
T_CONSOLE = "console"
T_VAR_var_initiable = "var_var_initiable"

def preprocessamento_linha(line_code):
    line_code = re.sub("[\\(\\=\\!\\[\\]\\)\\{\\}\\;\\?\\+\\-\\:\\&\\|\\,]", " \\g<0> ", line_code)#processa caracteres especiais
    line_code = re.sub(r"\.(?!\d)", " . ", line_code)#processa pontos quando não tem 1 número após ele

    return line_code

class Token():
    
    def __init__(self, tipo, valor=None):
        self.tipo = tipo
        self.valor = valor
        
    def __str__(self):
        return '<%s %s>' % (self.tipo, self.valor)
    
    def __repr__(self):
        return self.__str__()


class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def afd_keyword(token):
    keyword_list = ["this", ".", 'log', "constructor", "=>", "map", "slice", "reduce", "some", "sort", "toString", "String", "Boolean", "parseInt", "keys", "push", "pop", "forEach", "split", "join", "map", "then", "try", "catch", "[","]",  ",", ":"]
    return token in keyword_list

def afd_init(token):
    init_list = ['let', 'var', 'const']

    return token in init_list

def afd_function(token):
    return token in ['function']

def afd_while(token):
    return token in ['while']

def afd_if_else(token):
    return token in ['if', 'else']

def afd_class(token):
    return token in ['class']
    
def afd_return(token):
    return token in ['return']
    
def afd_console(token):
    return token in ['console']
    
def afd_int(token):
    try:
        token = int(token)
        return True
    except:
        return False
    
def afd_bool(token):
    return token in ['true', 'false']
    
def afd_float(token):
    try:
        token = float(token)
        return True
    except:
        return False
    
def afd_string(token):
    if token[0] == '"' and token[-1] == '"':
        if '"' not in token[1:-1]:
            return True
        else:
            raise ValueError('Aspas em um local inesperado.')
    else:
        return False
    
def afd_identificador(token):
    regex = re.compile('[a-zA-Z0-9_]+')
    r = regex.match(token)
    if r is not None:
        if r.group() == token:
            return True
        else:
            return False
    else:
        return False

def afd_delimiter(token):
    return token == ";"

def afd_operador(token):
    return token in "=+-*\\%<>&|"

def afd_block(token):
    return token in ["{", "}"]

def afd_var_type(token):
    return token in ["let", "var", "const"]

def afd_para(token):
    return token in ["(", ")"]
    
def afd_principal(token):
    if afd_keyword(token):
        return Token(T_KEYWORD, token)

    if afd_init(token):
        return Token(T_INIT, token)

    if afd_function(token):
        return Token(T_FUNCTION, token)

    if afd_class(token):
        return Token(T_CLASS, token)
    
    if afd_if_else(token):
        return Token(T_IF_ELSE, token)
    
    if afd_operador(token):
        return Token(T_OP, token)
    
    if afd_int(token):
        return Token(T_INT, token)
    
    if afd_bool(token):
        return Token(T_BOOL, token)
    
    if afd_float(token):
        return Token(T_FLOAT, token)
    
    if afd_string(token):
        return Token(T_STRING, token)
    
    if afd_console(token):
        return Token(T_CONSOLE, token)
    
    if afd_return(token):
        return Token(T_RETURN, token)
    
    if afd_delimiter(token):
        return Token(T_DELIMITER, token)
    
    if afd_block(token):
        return Token(T_BLOCK, token)
    
    if afd_while(token):
        return Token(T_WHILE, token)
    
    if afd_var_type(token):
        return Token(T_INIT, token)
    
    if afd_para(token):
        return Token(T_PARA, token)
    
    if afd_identificador(token):
        return Token(T_ID, token)
    
    raise ValueError('Valor inesperado')

class Parser():
    
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = -1
        self.token_atual = None
        self.token_proximo = None
        self.symbol_table = {}
        self.function_list = []
        self.class_list = []
        self.ir = []
        
        self.proximo()

    def generate_ir(self):
        return "\n".join(self.ir)
        
    def proximo(self):
        self.pos += 1
        
        if self.pos >= len(self.tokens):
            self.token_atual = Token(T_EOF)
            self.token_proximo = Token(T_EOF)
        elif self.pos + 1 >= len(self.tokens):
            self.token_atual = self.tokens[self.pos]
            self.token_proximo = Token(T_EOF)
        else:    
            self.token_atual = self.tokens[self.pos]
            self.token_proximo = self.tokens[self.pos+1]

        #print(self.token_atual)
        return self.token_atual
    
    
    def erro(self):
        raise Exception('Erro de sintaxe. %s' % (self.token_atual))
        
        
    def use(self, tipo, valor=None):
                
        if self.token_atual.tipo != tipo:
            self.erro()
        elif valor is not None and self.token_atual.valor != valor:
            self.erro()
        else:
            self.proximo()
    
    def instructions(self):
        """
        statements ::= <instruction> <instructions>
        """
        self.instruction()
        while self.token_atual.tipo in [T_INIT, T_ID, T_FUNCTION, T_CLASS, T_IF_ELSE, T_WHILE, T_CONSOLE]:
            self.instruction()
    
    def instruction(self):
        """
        instruction ::= <T_INIT> <id> ;
        instruction ::= statement ;
        """
        if self.token_atual.tipo == T_INIT:
            var_initiable = self.token_atual.valor

            if var_initiable not in ['let', 'const', 'var', ]:
                raise Exception(f"Erro semantico. Simbolo {var_initiable} não é um inicializador de variável.")

            self.use(T_INIT, var_initiable)
            name = self.token_atual.valor
            self.use(T_ID)
            if self.token_atual.valor == ";":
                self.use(T_DELIMITER, ";")
            if name in self.symbol_table:
                raise Exception(f"Erro semantico. Simbolo {name} foi declarado mais de uma vez.")
            self.symbol_table[name] = None
            self.ir.append(f"{var_initiable} {name};")
        else:
            self.statement()
            if self.token_atual.tipo == T_DELIMITER:
                self.use(T_DELIMITER, ";")
    
    def start(self):
        """
        start ::=  instructions 
        """
        self.ir.append("import React from 'react'")

        self.instructions()

    
    def return_optional(self):
        """
        return_optional ::=  return <expr> <delimeter ou Vazio> ou Vazio
        """
        if self.token_atual.tipo == T_RETURN:
            self.use(T_RETURN)
            self.ir.append(f"return")
            
            if self.token_atual.tipo != T_DELIMITER and self.token_atual.valor != '}':
                self.attr_statement()
                print("statement")

            if self.token_atual.tipo == T_DELIMITER:
                self.use(T_DELIMITER, ";")
                self.ir.append(f";")
                print(";")


    def statement(self): 
        """ statement ::= <id> <op => attr_statement """
        """ statement ::= <id> <op => <func_params> """
        """ statement ::= <id> <func_params> """
        """ statement ::= <func_creation> """
        """ statement ::= <class_creation> """
        """ statement ::= <if_statement> """
        """ statement ::= <attr_statement> """

        if self.token_atual.tipo == T_WHILE:
            return self.while_statement()            
        if self.token_atual.tipo == T_CONSOLE:
            return self.console_statement()            
        if self.token_atual.tipo == T_FUNCTION:
            return self.func_creation()            
        if self.token_atual.tipo == T_CLASS:
            return self.class_creation()
        if self.token_atual.tipo in [T_INT, T_STRING, T_FLOAT, T_BOOL, T_ID] and self.token_proximo.tipo in [T_OP,T_DELIMITER] and self.token_proximo.valor != "=":
            self.attr_statement()
            return
        if self.token_atual.valor == 'if':
            return self.if_statement()
        
        name = self.token_atual.valor

        if name in self.function_list:
            self.use(T_ID)
            return self.func_params()
        
        
        self.use(T_ID)
        self.use(T_OP, '=')
        self.ir.append(f"{name} =")

        self.attr_statement()

    def while_statement(self) -> any:
        """
        while_statement ::= <T_WHILE while> ( <attr_statement> ) { instructions }
        """

        self.use(T_WHILE)
        self.ir.append(f"while")

        self.use(T_PARA, '(')
        self.ir.append(f"(")
        
        self.attr_statement()
        
        self.use(T_PARA, ')')
        self.ir.append(f")")

        self.use(T_BLOCK, "{")
        self.ir.append("{")


        if self.token_atual.valor != '}':
            self.instructions()


        self.use(T_BLOCK, "}")
        self.ir.append("}")

    def console_statement(self) -> any:
        """
        console_statement ::= <T_CONSOLE consle> <keyword .> <keyword log> ( ATTR_STATEMENT )
        """

        self.use(T_CONSOLE)
        self.use(T_KEYWORD, '.')
        self.use(T_KEYWORD, 'log')
        self.use(T_PARA, '(')
        self.ir.append(f"console.log(")
        
        self.attr_statement()
        
        self.use(T_PARA, ')')
        self.ir.append(f")")

    def func_params(self) -> any:
        """
        func_params ::= ( <func_params_content ou Vazio> )
        """
        self.use(T_PARA, '(')
        self.ir.append(f"(")
        if self.token_atual.valor != ')':
            self.func_params_content()
        self.use(T_PARA, ')')
        self.ir.append(f")")

    def func_params_content(self) -> any:
        """
        func_params_content ::= <id ou string ou bool ou float ou int> <, <func_params_content> ou Vazio>
        """
        hasValue = self.token_atual.tipo in [T_INT, T_STRING, T_BOOL, T_FLOAT, T_ID]
        if hasValue:
            valueUsed = self.token_atual.valor
            self.use(self.token_atual.tipo)
            self.ir.append(f"{valueUsed}")

            hasContinuation = self.token_atual.valor == ','
            if hasContinuation:
                self.use(T_KEYWORD, ',')
                self.ir.append(f",")
                return self.func_params_content()
            
            return
        
        raise Exception(f"Erro semantico. Simbolo {self.token_atual.valor} não informado corretamente nos parâmetros de uma função.") 


        
    def func_creation(self) -> any:
        """
        func_creation ::= function ( func_variables ){ <return_optional> ou <instruction return_optional> }
        """
        self.use(T_FUNCTION)
        self.ir.append("function")

        functionName = self.token_atual.valor
        if functionName in self.function_list:
            raise Exception(f"Erro semantico. Função {functionName} foi declarado mais de uma vez.")
        if functionName in self.class_list:
            raise Exception(f"Erro semantico. Função {functionName} já foi declarado como uma classe.")
        self.function_list.append(functionName)
        self.symbol_table[functionName] = None
        self.use(T_ID)
        self.ir.append(f" {functionName}")

        self.use(T_PARA, "(")
        self.ir.append(f"(")

        self.func_variables(functionName)

        self.use(T_PARA, ")")
        self.ir.append(f")")

        self.use(T_BLOCK, "{")
        self.ir.append("{")


        if self.token_atual.tipo == T_RETURN:
            self.return_optional()
        else:
            self.instructions()
            self.return_optional()


        self.use(T_BLOCK, "}")
        self.ir.append("}")

        
    def func_variables(self, functionName) -> any:
        """
        func_variables ::= id <func_variables_continuacao>
        """
        idName = self.token_atual.valor
        self.symbol_table[idName] = 0 #adicionar lógica de escopo de variável caso esteja dentro de uma função
        self.use(T_ID)
        self.ir.append(f"{idName}")
        self.func_variables_continuacao(functionName)

    def func_variables_continuacao(self,functionName) -> any:
        """
        func_variables_continuacao ::= <func_operators> <func_comma> ou Vazio
        """
        
        hasOperators = self.token_atual.valor in ['=']
        hasComma = self.token_atual.tipo in [T_KEYWORD] and self.token_atual.valor == ","
        if hasOperators or hasComma:
            self.func_operators()
            self.func_comma(functionName)
            return

    def func_operators(self) -> any:
        """
        func_operators ::= <id ou int ou string ou float ou boleano> ou Vazio
        """
        hasOperators = self.token_atual.valor in ['=']
        if hasOperators:
            self.use(T_OP, '=')
            
            variableValue = self.token_atual.valor

            hasInvalidType = self.token_atual.tipo not in [T_INT, T_STRING, T_FLOAT, T_BOOL, T_ID]
            if hasInvalidType:
                raise Exception(f"Erro semantico. Valor {self.token_atual.valor} não é válido dentro de uma variável de função.")
                
            self.use(self.token_atual.tipo)
            self.ir.append(f"= {variableValue}")

    def func_comma(self, functionName) -> any:
        """
        func_comma ::= , <func_variables> ou Vazio
        """
        hasComma = self.token_atual.tipo in [T_KEYWORD] and self.token_atual.valor == ","
        if hasComma:
            self.use(T_KEYWORD, ',')
            self.ir.append(f",")
            self.func_variables(functionName)


    def class_creation(self) -> any:
        """
        class_creation ::= class  
        """
        self.use(T_CLASS)
        self.ir.append("class")


        className = self.token_atual.valor
        if className in self.class_list:
            raise Exception(f"Erro semantico. Classe {className} foi declarado mais de uma vez.")
        if className in self.function_list:
            raise Exception(f"Erro semantico. Classe {className} já foi declarado como uma função.")
        self.class_list.append(className)
        self.symbol_table[className] = None
        self.use(T_ID)
        self.ir.append(f"{className}")

        self.use(T_BLOCK, "{")
        self.ir.append("{")


        self.class_content(className)

        self.use(T_BLOCK, "}")
        self.ir.append("}")

    def class_content(self, className) -> any:
        """
        class_content ::= constructor ( <construct_params> ) { <instructions> }
        class_content ::= instruction 
        class_content ::= Vazio 
        """
        if self.token_atual.valor == '}':
            return
        
        if self.token_atual.valor == 'constructor':
            self.use(T_KEYWORD, "constructor")
            self.ir.append("constructor")

            self.use(T_PARA, "(")
            self.ir.append("(")

            self.construct_params(className)

            self.use(T_PARA, ")")
            self.ir.append(")")

            self.use(T_BLOCK, "{")
            self.ir.append("{")

            if self.token_atual.valor == '}':
                self.use(T_BLOCK, "}")
                self.ir.append("}")
            else: 
                self.instructions()

            return

        self.instructions()

    def construct_params(self, className) -> any:
        """
        construct_params -> id C_OP VIRGULA ou Vazio
        C_OP -> = <int ou string ou float ou boleano> ou Vazio
        VIRGULA -> , <construct_params> ou Vazio
        """

        if self.token_atual.tipo == T_ID:
            idName = self.token_atual.valor
            self.use(T_ID)
            self.ir.append(f"{idName}")
            self.symbol_table[idName] = None

            #C_OP
            if self.token_atual.valor == "=":
                self.use(T_OP, '=')
                variableValue = self.token_atual.valor

                hasInvalidType = self.token_atual.tipo not in [T_INT, T_STRING, T_FLOAT, T_BOOL, T_ID]
                if hasInvalidType:
                    raise Exception(f"Erro semantico. Valor {self.token_atual.valor} não é válido dentro de uma variável de constructor.")
                    
                self.use(self.token_atual.tipo)
                self.ir.append(f"= {variableValue}")

            #VIRGULA
            if self.token_atual.valor == ",":
                self.use(T_KEYWORD, ',')
                self.ir.append(f",")

                return self.construct_params(className)


    def attr_statement(self) -> any:
        """
        attr_statement ::= <id ou string ou int ou float ou boolean> <IF_CONDITION_CONTINUES>
        """

        valorAtual = self.token_atual.valor
        tipoAtual = self.token_atual.tipo

        hasInvalidType = tipoAtual not in [T_INT, T_STRING, T_FLOAT, T_BOOL, T_ID]
        if hasInvalidType:
            raise Exception(f"Erro semantico. Valor {valorAtual} não é válido dentro de uma condição de if.")

        isIdAndIsNotDeclared = tipoAtual == T_ID and valorAtual not in self.symbol_table
        if isIdAndIsNotDeclared:
            raise Exception(f"Erro semantico. Valor {valorAtual} não foi declarado ainda.")

        isArgumentFunction = valorAtual in self.function_list
        if isArgumentFunction:
            self.ir.append(f"{self.token_atual.valor}")
            self.use(T_ID)
            return self.func_params()

        
        self.use(tipoAtual)
        self.ir.append(f"{valorAtual}")
        self.attr_statement_continues()

    

    def attr_statement_continues(self) -> any:
        """
        attr_statement_continues ::= <OP se for +-*/<> será 1 vez, caso contrário são duas instâncias> <IF_CONDITION> ou Vazio
        """
        if self.token_atual.tipo == T_OP:
            opValue = self.token_atual.valor

            justOneOccurrence = ['+','-','*','/']
            canHaveTwoOrOne = ['>','<','!']
            justTwoOccurrences = ['&','|', '=']

            if opValue in justOneOccurrence:
                self.use(T_OP)
                self.ir.append(f"{opValue}")
                return self.attr_statement()

            if opValue in canHaveTwoOrOne:
                self.use(T_OP)
                opValueGroup = f"{opValue}"

                opValue = self.token_atual.valor
                

                #os campos mapeados no canHaveTwoOrOne, sempre possuem o = como sequência
                if opValue == '=':
                    self.use(T_OP)
                    opValueGroup = f"{opValueGroup}{opValue}"
                    self.ir.append(f"{opValueGroup}")
                    return self.attr_statement()

                self.ir.append(f"{opValueGroup}")
                return self.attr_statement()

                

            if opValue in justTwoOccurrences:
                self.use(T_OP)
                opValueGroup = f"{opValue}"

                opValue = self.token_atual.valor
                opValueGroup = f"{opValueGroup}{opValue}"
            
                self.use(T_OP)
                self.ir.append(f"{opValueGroup}")
                return self.attr_statement()

            if opValue not in [')']:
                raise Exception(f"Erro semantico. Operador {opValue} não foi mapeado.")

    def if_statement(self) -> any:
        """
        if_statement ::= <keyword if> ( IF_CONDITION ) IF_CONTENT ou Vazio
        """
        if self.token_atual.valor == 'if':
            self.use(T_IF_ELSE, 'if')
            self.ir.append(f"if")

            self.use(T_PARA, "(")
            self.ir.append("(")

            self.attr_statement()

            self.use(T_PARA, ")")
            self.ir.append(")")

            self.if_content()


    
    def if_content(self) -> any:
        """
        if_content ::= { <instructions> } <ELSE ou ELSEIF>
        """
        self.use(T_BLOCK, "{")
        self.ir.append("{")

        if self.token_atual.valor != '}':
            self.instructions()

        self.use(T_BLOCK, "}")
        self.ir.append("}")

        if self.token_atual.valor == 'else' and self.token_proximo.valor == 'if':
            self.else_if_statement()
        elif self.token_atual.valor == 'else':
            self.else_statement()

    def else_statement(self) -> any:
        """
        else_statement ::= else { <instructions> } ou Vazio
        """
        if self.token_atual.valor == 'else':
            self.use(T_IF_ELSE, 'else')
            self.use(T_BLOCK, "{")
            self.ir.append("else {")

            if self.token_atual.valor != '}':
                self.instructions()

            self.use(T_BLOCK, "}")
            self.ir.append("}")

    def else_if_statement(self) -> any:
        """
        else_if_statement ::= else if <if_statement> ou Vazio
        """

        if self.token_atual.valor == 'else' and self.token_proximo.valor == 'if':
            self.use(T_IF_ELSE, 'else')
            self.ir.append(f"else")

            self.if_statement()



    def expr(self) -> any:
        """
        expr ::= term ( <op +> | <op -> term )*
        """

        t, s = self.expr_t()
        res, s2 = self.expr_e_line(t)
        return res, f"{s} {s2}"

    def expr_t(self) -> any:
        """
        expr_t ::= expr_f expr_t_line
        """
        r, s = self.expr_f()
        r2, s2 = self.expr_t_line(r)
        return r2, f"{s} {s2}"

    def expr_e_line(self, inherited_t: any):
        """
        expr_e_line ::= <op +> expr_t expr_e_line | epsilon
        """
        if self.token_atual.tipo == T_OP and self.token_atual.valor == "+":
            self.use(T_OP, "+")
            a, s = self.expr_t()
            res = a + inherited_t
            a2, s2 = self.expr_e_line(res)
            str_expr = f"+ {s} {s2}"
            return a2, str_expr
        # Prod vazia
        return inherited_t, ""

    def expr_t_line(self, inherited_t: any):
        """
        expr_e_line ::= <op *> expr_f expr_t_line | epsilon
        """
        if self.token_atual.tipo == T_OP and self.token_atual.valor == "*":
            self.use(T_OP, "*")
            a, s = self.expr_f()
            res = a * inherited_t
            a2, s2 = self.expr_t_line(res)
            str_expr = f"* {s} {s2}"
            return a2, str_expr
        # Prod vazia
        return inherited_t, ""

    def expr_f(self):
        """
        expr_f ::= ( expr ) | <id> | <int> | <float> | <string>
        """
        str_expr = ""
        if self.token_atual.tipo == T_PARA:
            self.use(T_PARA, "(")
            res, s = self.expr()
            self.use(T_PARA, ")")
            str_expr = f"({s})"
        elif self.token_atual.tipo == T_ID:
            if self.token_atual.valor not in self.symbol_table:
                raise Exception(f"Erro Semantico, variavel {self.token_atual.valor} nao foi declarada")
            res = self.symbol_table[self.token_atual.valor]
            str_expr = self.token_atual.valor
            self.use(T_ID)
        elif self.token_atual.tipo == T_INT:
            str_expr = str(self.token_atual.valor)
            res = int(self.token_atual.valor)
            self.use(T_INT)
        elif self.token_atual.tipo == T_FLOAT:
            str_expr = str(self.token_atual.valor)
            res = float(self.token_atual.valor)
            self.use(T_FLOAT)
        elif self.token_atual.tipo == T_BOOL:
            str_expr = str(self.token_atual.valor)
            res = str(self.token_atual.valor)
            self.use(T_BOOL)
        elif self.token_atual.tipo == T_STRING:
            str_expr = self.token_atual.valor
            res = self.token_atual.valor
            self.use(T_STRING)
        else:
            self.erro()

        return res, str_expr


arquivo = open('codigo.js','r')
ln = 1

tokens = []

for l in arquivo.readlines():
    # analisador lexico
    l = l.replace('\n','') # remove a quebra de linha
    l = preprocessamento_linha(l)

    for token in l.split():        
        try:
            tokens.append(afd_principal(token))
        except Exception as e:
            print(tokens)
            print(str(e) + " na posição %i da linha %i - %s" % (l.index(token), ln, token))
            raise StopExecution
    ln += 1

#print([str(t) for t in tokens])

# analisador sintatico

parser = Parser(tokens)
parser.start()
code = parser.generate_ir()

with open("codigo.jsx", "w") as out_f:
    out_f.write(code)

statement
;
